# Data Analysis & Visualization

This notebook provides comprehensive analysis and professional visualizations for your research report.

## Objectives
1. Aggregate all experimental data
2. Generate statistical insights
3. Create publication-quality visualizations
4. Provide data for report writing

In [ ]:
# Setup
import sys
sys.path.append('../src')

from database import HallucinationDB
from config import Config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Libraries loaded")

## Load All Data

In [ ]:
# Connect to database
db = HallucinationDB()

# Get all experiments
df_experiments = db.get_all_experiments()
print("Experiments Summary:")
print(df_experiments)

# Get overall statistics
stats = db.get_statistics()
print(f"\nTotal experiments: {stats['total_experiments']}")
print(f"Total tests conducted: {stats['total_tests']}")

print("\nHallucination rates by mitigation strategy:")
print(stats['hallucination_by_strategy'])

In [ ]:
# Load all experimental results into one dataframe
all_results = []

for idx, exp in df_experiments.iterrows():
    exp_id = exp['experiment_id']
    df_exp = db.get_experiment_results(exp_id)
    all_results.append(df_exp)

if all_results:
    df_all = pd.concat(all_results, ignore_index=True)
    print(f"\nTotal data points: {len(df_all)}")
    print(f"Date range: {df_all['created_at'].min()} to {df_all['created_at'].max()}")
else:
    print("\n⚠️  No experimental data found. Please run the testing notebooks first.")
    df_all = pd.DataFrame()

## 1. Overall Hallucination Rates

### 1.1 By Mitigation Strategy

In [ ]:
if not df_all.empty:
    # Calculate rates by strategy
    strategy_performance = df_all.groupby('mitigation_strategy').agg({
        'is_hallucination': ['count', 'sum', 'mean'],
        'response_time_ms': 'mean',
        'tokens_used': 'mean'
    }).round(3)
    
    strategy_performance.columns = ['Total Tests', 'Hallucinations', 'Hallucination Rate', 
                                    'Avg Response Time (ms)', 'Avg Tokens']
    # Ensure numeric type for Rate (%) column
    strategy_performance['Hallucination Rate (%)'] = (strategy_performance['Hallucination Rate'] * 100).astype(float).round(2)
    
    print("Strategy Performance Metrics:")
    print(strategy_performance)
    
    # Save to CSV
    strategy_performance.to_csv('../results/reports/strategy_performance.csv')
    print("\n✓ Saved to results/reports/strategy_performance.csv")

In [ ]:
# Professional Visualization: Hallucination Rate Comparison
if not df_all.empty:
    # Set modern style (compatible with all seaborn versions)
    try:
        plt.style.use('seaborn-v0_8-whitegrid')
    except:
        try:
            plt.style.use('seaborn-whitegrid')
        except:
            plt.style.use('ggplot')
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    strategies = strategy_performance.index
    rates = strategy_performance['Hallucination Rate (%)'].values
    
    # Modern, professional color palette
    colors = ['#264653', '#2A9D8F', '#E9C46A', '#F4A261', '#E76F51']
    
    bars = ax.barh(range(len(strategies)), rates, color=colors[:len(strategies)], 
                   alpha=0.85, edgecolor='black', linewidth=1.5)
    
    ax.set_yticks(range(len(strategies)))
    ax.set_yticklabels([s.replace('_', ' ').title() for s in strategies], fontsize=12, fontweight='600')
    ax.set_xlabel('Hallucination Rate (%)', fontsize=13, fontweight='bold')
    ax.set_title('Hallucination Rate by Mitigation Strategy', 
                 fontsize=16, fontweight='bold', pad=20)
    ax.grid(axis='x', alpha=0.3, linestyle='--', linewidth=0.8)
    ax.set_axisbelow(True)
    
    # Add value labels on bars
    for i, (bar, val) in enumerate(zip(bars, rates)):
        width = bar.get_width()
        ax.text(width + max(rates)*0.01, bar.get_y() + bar.get_height()/2.,
               f'{val:.1f}%', ha='left', va='center', fontweight='bold', fontsize=11)
    
    # Remove top and right spines for cleaner look
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.tight_layout()
    plt.savefig('../results/charts/hallucination_rate_by_strategy.png', 
                dpi=300, bbox_inches='tight', facecolor='white')
    plt.show()
    
    print("✓ Chart saved to results/charts/hallucination_rate_by_strategy.png")

### 1.2 By Prompt Category

In [ ]:
if not df_all.empty:
    # Category performance
    category_performance = df_all.groupby('prompt_category').agg({
        'is_hallucination': ['count', 'sum', 'mean']
    }).round(3)
    
    category_performance.columns = ['Total', 'Hallucinations', 'Rate']
    # Ensure numeric type for Rate (%) column
    category_performance['Rate (%)'] = (category_performance['Rate'] * 100).astype(float).round(2)
    category_performance = category_performance.sort_values('Rate (%)', ascending=False)
    
    print("Hallucination Rate by Prompt Category:")
    print(category_performance)

In [ ]:
# ============================================================================
# CATEGORY vs STRATEGY HEATMAP ANALYSIS
# ============================================================================

if not df_all.empty:
    
    print("\n" + "="*80)
    print("HALLUCINATION RATE HEATMAP: CATEGORY vs STRATEGY")
    print("="*80)
    
    # Create pivot table with proper type conversion
    pivot = df_all.pivot_table(
        values='is_hallucination',
        index='prompt_category',
        columns='mitigation_strategy',
        aggfunc='mean'
    )
    
    # Ensure numeric type and convert to percentage
    pivot = pivot.astype(float) * 100
    
    print(f"\n📊 Heatmap Data:")
    print(f"   ✅ Prompt categories: {len(pivot.index)}")
    print(f"   ✅ Strategies: {len(pivot.columns)}")
    print(f"   ✅ Data points: {pivot.size}")
    
    # ============================================================================
    # PLOTLY INTERACTIVE HEATMAP
    # ============================================================================
    
    print(f"\n🔨 Creating interactive heatmap...")
    
    # Prepare data for Plotly
    z_data = pivot.values
    x_labels = [col.replace('_', ' ').title() for col in pivot.columns]
    y_labels = [idx.replace('_', ' ').title() for idx in pivot.index]
    
    # Create heatmap
    fig = go.Figure(data=go.Heatmap(
        z=z_data,
        x=x_labels,
        y=y_labels,
        colorscale='RdYlGn_r',  # Red (high) to Green (low)
        zmid=50,  # Center at 50%
        zmin=0,
        zmax=100,
        text=np.round(z_data, 1),
        texttemplate='<b>%{text:.1f}%</b>',
        textfont=dict(size=11, family='Arial Black', color='#2c3e50'),
        hovertemplate='<b>%{y}</b><br><b>%{x}</b><br>Hallucination Rate: %{z:.1f}%<extra></extra>',
        colorbar=dict(
            title=dict(
                text='<b>Hallucination<br>Rate (%)</b>',
                font=dict(size=13, family='Arial')
            ),
            tickfont=dict(size=11, family='Arial'),
            len=0.7,
            thickness=20
        )
    ))
    
    fig.update_layout(
        title=dict(
            text='<b>Hallucination Rate Heatmap: Category vs Strategy</b><br><sup>Green = Low hallucinations | Red = High hallucinations</sup>',
            x=0.5,
            xanchor='center',
            font=dict(size=20, color='#2c3e50', family='Arial Black')
        ),
        xaxis=dict(
            title='<b>Mitigation Strategy</b>',
            titlefont=dict(size=14, family='Arial', color='#2c3e50'),
            tickfont=dict(size=11, family='Arial', color='#2c3e50'),
            side='bottom'
        ),
        yaxis=dict(
            title='<b>Prompt Category</b>',
            titlefont=dict(size=14, family='Arial', color='#2c3e50'),
            tickfont=dict(size=11, family='Arial', color='#2c3e50')
        ),
        height=max(600, len(y_labels) * 50),  # Dynamic height based on categories
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa',
        font=dict(family='Arial', size=12, color='#2c3e50'),
        margin=dict(t=100, b=80, l=150, r=120)
    )
    
    # Save and show
    fig.write_html('../results/charts/category_strategy_heatmap_interactive.html')
    fig.show()
    
    print(f"   ✅ Heatmap created successfully")
    
    # ============================================================================
    # HEATMAP INSIGHTS
    # ============================================================================
    
    print("\n" + "="*80)
    print("HEATMAP INSIGHTS")
    print("="*80)
    
    # Find highest and lowest hallucination rates
    print(f"\n🔥 Highest Hallucination Combinations:")
    max_values = []
    for strategy in pivot.columns:
        for category in pivot.index:
            if not pd.isna(pivot.loc[category, strategy]):
                max_values.append((category, strategy, pivot.loc[category, strategy]))
    max_values.sort(key=lambda x: x[2], reverse=True)
    
    for i, (cat, strat, rate) in enumerate(max_values[:5], 1):
        print(f"   {i}. {cat.replace('_', ' ').title():30s} + {strat.replace('_', ' ').title():20s} = {rate:5.1f}%")
    
    print(f"\n✅ Lowest Hallucination Combinations:")
    min_values = [x for x in max_values if x[2] < 50]  # Only show meaningful low values
    min_values.sort(key=lambda x: x[2])
    
    for i, (cat, strat, rate) in enumerate(min_values[:5], 1):
        print(f"   {i}. {cat.replace('_', ' ').title():30s} + {strat.replace('_', ' ').title():20s} = {rate:5.1f}%")
    
    # Strategy effectiveness across categories
    print(f"\n📊 Strategy Effectiveness (Average across all categories):")
    for strategy in pivot.columns:
        avg_rate = pivot[strategy].mean()
        print(f"   {strategy.replace('_', ' ').title():20s}: {avg_rate:5.1f}%")
    
    # Category vulnerability
    print(f"\n⚠️  Category Vulnerability (Average across all strategies):")
    for category in pivot.index:
        avg_rate = pivot.loc[category].mean()
        print(f"   {category.replace('_', ' ').title():30s}: {avg_rate:5.1f}%")
    
    print(f"\n📁 Output Files:")
    print(f"   ✅ Interactive HTML: ../results/charts/category_strategy_heatmap_interactive.html")
    
    print("\n" + "="*80)

## 2. Cost-Benefit Analysis

### 2.1 Token Usage vs Accuracy

In [ ]:
# ============================================================================
# COST-BENEFIT ANALYSIS
# ============================================================================

if not df_all.empty:
    
    print("\n" + "="*80)
    print("COST-BENEFIT ANALYSIS: TOKEN USAGE & RESPONSE TIME vs ACCURACY")
    print("="*80)
    
    # Calculate accuracy (inverse of hallucination rate)
    strategy_costs = df_all.groupby('mitigation_strategy').agg({
        'tokens_used': 'mean',
        'is_hallucination': 'mean',
        'response_time_ms': 'mean'
    })
    
    strategy_costs['accuracy'] = (1 - strategy_costs['is_hallucination']) * 100
    strategy_costs['hallucination_rate'] = strategy_costs['is_hallucination'] * 100
    
    print(f"\n📊 Strategy Performance Metrics:")
    for strategy in strategy_costs.index:
        row = strategy_costs.loc[strategy]
        print(f"   {strategy.replace('_', ' ').title():20s} | Tokens: {row['tokens_used']:6.0f} | Time: {row['response_time_ms']:6.0f}ms | Accuracy: {row['accuracy']:5.1f}%")
    
    # ============================================================================
    # PLOTLY INTERACTIVE SCATTER PLOTS
    # ============================================================================
    
    print(f"\n🔨 Creating interactive cost-benefit analysis charts...")
    
    # Color scheme matching notebook 03
    colors_dict = {
        'Baseline': '#34495e',
        'Rag': '#27ae60',
        'Constitutional Ai': '#e74c3c',
        'Chain Of Thought': '#3498db'
    }
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            '<b>💰 Token Cost vs Accuracy</b><br><sub>(Top-Left = Best Value)</sub>',
            '<b>⚡ Speed vs Accuracy</b><br><sub>(Top-Left = Best Performance)</sub>'
        ),
        horizontal_spacing=0.12
    )
    
    # 1. TOKENS VS ACCURACY
    for idx, strategy in enumerate(strategy_costs.index):
        strategy_title = strategy.replace('_', ' ').title()
        color = colors_dict.get(strategy_title, '#34495e')
        
        # Size based on performance (smaller tokens/time = larger marker)
        size = 30 if strategy_costs.loc[strategy, 'tokens_used'] < 500 else (25 if strategy_costs.loc[strategy, 'tokens_used'] < 800 else 20)
        
        fig.add_trace(
            go.Scatter(
                x=[strategy_costs.loc[strategy, 'tokens_used']],
                y=[strategy_costs.loc[strategy, 'accuracy']],
                mode='markers+text',
                marker=dict(
                    size=size,
                    color=color,
                    line=dict(color='#2c3e50', width=3),
                    opacity=0.85
                ),
                text=[strategy_title],
                textposition='bottom center',
                textfont=dict(size=12, color='#2c3e50', family='Arial Black'),
                hovertemplate=f"<b>{strategy_title}</b><br>Tokens: {strategy_costs.loc[strategy, 'tokens_used']:.0f}<br>Accuracy: {strategy_costs.loc[strategy, 'accuracy']:.1f}%<br>Hall. Rate: {strategy_costs.loc[strategy, 'hallucination_rate']:.1f}%<extra></extra>",
                showlegend=False,
                name=strategy_title
            ),
            row=1, col=1
        )
    
    # 2. RESPONSE TIME VS ACCURACY
    for idx, strategy in enumerate(strategy_costs.index):
        strategy_title = strategy.replace('_', ' ').title()
        color = colors_dict.get(strategy_title, '#34495e')
        
        size = 30 if strategy_costs.loc[strategy, 'response_time_ms'] < 1000 else (25 if strategy_costs.loc[strategy, 'response_time_ms'] < 3000 else 20)
        
        fig.add_trace(
            go.Scatter(
                x=[strategy_costs.loc[strategy, 'response_time_ms']],
                y=[strategy_costs.loc[strategy, 'accuracy']],
                mode='markers+text',
                marker=dict(
                    size=size,
                    color=color,
                    line=dict(color='#2c3e50', width=3),
                    opacity=0.85
                ),
                text=[strategy_title],
                textposition='bottom center',
                textfont=dict(size=12, color='#2c3e50', family='Arial Black'),
                hovertemplate=f"<b>{strategy_title}</b><br>Time: {strategy_costs.loc[strategy, 'response_time_ms']:.0f}ms<br>Accuracy: {strategy_costs.loc[strategy, 'accuracy']:.1f}%<br>Hall. Rate: {strategy_costs.loc[strategy, 'hallucination_rate']:.1f}%<extra></extra>",
                showlegend=False,
                name=strategy_title
            ),
            row=1, col=2
        )
    
    # Update layout
    fig.update_xaxes(title_text="<b>Average Tokens Used</b><br>(Lower = Cheaper)", row=1, col=1, showgrid=True, gridcolor='#ecf0f1', gridwidth=1)
    fig.update_xaxes(title_text="<b>Average Response Time (ms)</b><br>(Lower = Faster)", row=1, col=2, showgrid=True, gridcolor='#ecf0f1', gridwidth=1)
    fig.update_yaxes(title_text="<b>Accuracy %</b><br>(Higher = Better)", row=1, col=1, showgrid=True, gridcolor='#ecf0f1', gridwidth=1)
    fig.update_yaxes(title_text="<b>Accuracy %</b><br>(Higher = Better)", row=1, col=2, showgrid=True, gridcolor='#ecf0f1', gridwidth=1)
    
    fig.update_layout(
        title=dict(
            text='<b>Cost-Benefit Analysis: Performance Trade-offs</b><br><sup>Ideal = Top-Left Corner (High Accuracy + Low Cost/Time)</sup>',
            x=0.5,
            xanchor='center',
            font=dict(size=22, color='#2c3e50', family='Arial Black')
        ),
        height=600,
        showlegend=False,
        plot_bgcolor='white',
        paper_bgcolor='#f8f9fa',
        font=dict(family='Arial', size=12, color='#2c3e50'),
        margin=dict(t=120, b=80, l=80, r=80)
    )
    
    # Save and show
    fig.write_html('../results/charts/cost_benefit_analysis_interactive.html')
    fig.show()
    
    print(f"   ✅ Charts created successfully")
    
    # ============================================================================
    # COST-BENEFIT SUMMARY
    # ============================================================================
    
    print("\n" + "="*80)
    print("COST-BENEFIT SUMMARY")
    print("="*80)
    
    # Find best performers
    best_accuracy = strategy_costs['accuracy'].idxmax()
    cheapest = strategy_costs['tokens_used'].idxmin()
    fastest = strategy_costs['response_time_ms'].idxmin()
    
    print(f"\n🎯 Best Performers:")
    print(f"   🥇 HIGHEST ACCURACY: {best_accuracy.replace('_', ' ').title():20s} ({strategy_costs.loc[best_accuracy, 'accuracy']:.1f}%)")
    print(f"   💰 CHEAPEST:         {cheapest.replace('_', ' ').title():20s} ({strategy_costs.loc[cheapest, 'tokens_used']:.0f} tokens)")
    print(f"   ⚡ FASTEST:          {fastest.replace('_', ' ').title():20s} ({strategy_costs.loc[fastest, 'response_time_ms']:.0f}ms)")
    
    # Trade-off analysis
    print(f"\n⚖️  Trade-off Analysis:")
    for strategy in strategy_costs.index:
        row = strategy_costs.loc[strategy]
        # Calculate normalized scores (higher is better)
        norm_acc = row['accuracy'] / 100
        norm_cost = 1 - (row['tokens_used'] / strategy_costs['tokens_used'].max())
        norm_speed = 1 - (row['response_time_ms'] / strategy_costs['response_time_ms'].max())
        overall_score = (norm_acc + norm_cost + norm_speed) / 3
        
        print(f"   {strategy.replace('_', ' ').title():20s} | Overall Score: {overall_score:.3f}")
        print(f"      Accuracy: {norm_acc:.3f} | Cost Eff: {norm_cost:.3f} | Speed: {norm_speed:.3f}")
    
    print(f"\n📁 Output Files:")
    print(f"   ✅ Interactive HTML: ../results/charts/cost_benefit_analysis_interactive.html")
    
    print("\n" + "="*80)

## 3. Severity Analysis

In [ ]:
# ============================================================================
# HALLUCINATION SEVERITY ANALYSIS
# ============================================================================

if not df_all.empty and 'severity' in df_all.columns:
    
    print("\n" + "="*80)
    print("HALLUCINATION SEVERITY DISTRIBUTION ANALYSIS")
    print("="*80)
    
    # Filter only hallucinations
    df_hallucinations = df_all[df_all['is_hallucination'] == True]
    
    if len(df_hallucinations) > 0:
        
        print(f"\n📊 Severity Analysis:")
        print(f"   ✅ Total hallucinations detected: {len(df_hallucinations)}")
        print(f"   ✅ Strategies analyzed: {df_hallucinations['mitigation_strategy'].nunique()}")
        
        # Severity distribution
        severity_dist = df_hallucinations.groupby(['mitigation_strategy', 'severity']).size().unstack(fill_value=0)
        
        print(f"\n🎯 Severity Breakdown by Strategy:")
        for strategy in severity_dist.index:
            total = severity_dist.loc[strategy].sum()
            print(f"   {strategy.replace('_', ' ').title():20s} - {total} hallucinations")
            for severity in severity_dist.columns:
                count = severity_dist.loc[strategy, severity]
                pct = (count / total * 100) if total > 0 else 0
                print(f"      {severity.title():10s}: {count:3d} ({pct:5.1f}%)")
        
        # ========================================================================
        # PLOTLY INTERACTIVE STACKED BAR CHART
        # ========================================================================
        
        print(f"\n🔨 Creating interactive severity distribution chart...")
        
        # Color scheme for severity levels
        severity_colors = {
            'low': '#27ae60',
            'medium': '#f39c12', 
            'high': '#e67e22',
            'critical': '#c0392b'
        }
        
        fig = go.Figure()
        
        # Add a trace for each severity level
        for severity in severity_dist.columns:
            fig.add_trace(go.Bar(
                name=severity.title(),
                x=[s.replace('_', ' ').title() for s in severity_dist.index],
                y=severity_dist[severity].values,
                marker=dict(
                    color=severity_colors.get(severity, '#95a5a6'),
                    line=dict(color='#2c3e50', width=2),
                    opacity=0.85
                ),
                text=severity_dist[severity].values,
                textposition='inside',
                textfont=dict(size=13, color='white', family='Arial Black'),
                hovertemplate='<b>%{x}</b><br>' + severity.title() + ': %{y}<extra></extra>'
            ))
        
        fig.update_layout(
            title=dict(
                text='<b>Hallucination Severity Distribution by Strategy</b><br><sup>Higher bars = More severe hallucinations</sup>',
                x=0.5,
                xanchor='center',
                font=dict(size=20, color='#2c3e50', family='Arial Black')
            ),
            xaxis=dict(
                title='<b>Mitigation Strategy</b>',
                titlefont=dict(size=14, family='Arial', color='#2c3e50'),
                showgrid=False
            ),
            yaxis=dict(
                title='<b>Number of Hallucinations</b>',
                titlefont=dict(size=14, family='Arial', color='#2c3e50'),
                showgrid=True,
                gridcolor='#ecf0f1',
                gridwidth=1
            ),
            barmode='stack',
            height=600,
            showlegend=True,
            legend=dict(
                title=dict(text='<b>Severity Level</b>', font=dict(size=13)),
                x=0.85,
                y=0.98,
                orientation='v',
                font=dict(size=12, family='Arial', color='#2c3e50'),
                bgcolor='rgba(255,255,255,0.9)',
                bordercolor='#2c3e50',
                borderwidth=2
            ),
            plot_bgcolor='white',
            paper_bgcolor='#f8f9fa',
            font=dict(family='Arial', size=12, color='#2c3e50'),
            margin=dict(t=100, b=80, l=80, r=120)
        )
        
        # Save and show
        os.makedirs('../results/charts', exist_ok=True)
        fig.write_html('../results/charts/severity_distribution_interactive.html')
        fig.show()
        
        print(f"   ✅ Chart created successfully")
        
        # ========================================================================
        # SUMMARY
        # ========================================================================
        
        print("\n" + "="*80)
        print("SEVERITY DISTRIBUTION SUMMARY")
        print("="*80)
        
        # Find most severe strategy
        if 'critical' in severity_dist.columns:
            critical_counts = severity_dist['critical']
            most_critical = critical_counts.idxmax()
            print(f"\n⚠️  Most Critical Hallucinations:")
            print(f"   Strategy: {most_critical.replace('_', ' ').title()}")
            print(f"   Critical count: {critical_counts[most_critical]}")
        
        # Overall severity summary
        total_by_severity = severity_dist.sum()
        print(f"\n📊 Overall Severity Distribution:")
        for severity in total_by_severity.index:
            count = total_by_severity[severity]
            pct = (count / total_by_severity.sum() * 100)
            print(f"   {severity.title():10s}: {count:3d} ({pct:5.1f}%)")
        
        print(f"\n📁 Output Files:")
        print(f"   ✅ Interactive HTML: ../results/charts/severity_distribution_interactive.html")
        
        print("\n" + "="*80)
        
    else:
        print("\n✓ No hallucinations detected - all strategies performing perfectly!")
else:
    print("\n⚠️  Severity data not available in dataset")

## 4. Interactive Visualizations (Plotly)

In [ ]:
if not df_all.empty:
    # Prepare data with clean labels
    df_plot = df_all.copy()
    df_plot['Strategy'] = df_plot['mitigation_strategy'].str.replace('_', ' ').str.title()
    df_plot['Category'] = df_plot['prompt_category'].str.replace('_', ' ').str.title()
    df_plot['Hallucinated'] = df_plot['is_hallucination'].map({True: 'Yes', False: 'No'})
    
    # Interactive scatter: Response time vs Tokens, colored by hallucination
    fig = px.scatter(df_plot, 
                    x='tokens_used', 
                    y='response_time_ms',
                    color='Hallucinated',
                    facet_col='Strategy',
                    facet_col_wrap=2,  # Wrap to 2 columns to prevent overcrowding
                    hover_data={'Category': True, 
                               'severity': True,
                               'tokens_used': ':.0f',
                               'response_time_ms': ':.1f',
                               'Strategy': False,
                               'Hallucinated': False},
                    title='<b>Response Characteristics by Mitigation Strategy</b>',
                    labels={'tokens_used': 'Tokens Used',
                           'response_time_ms': 'Response Time (ms)',
                           'Hallucinated': 'Hallucination'},
                    color_discrete_map={'Yes': '#E74C3C', 'No': '#2ECC71'},
                    height=600)
    
    # Update layout for better readability
    fig.update_layout(
        font=dict(size=12, family='Arial, sans-serif'),
        title_font=dict(size=18, family='Arial, sans-serif'),
        plot_bgcolor='rgba(240, 240, 240, 0.5)',
        paper_bgcolor='white',
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Arial, sans-serif"
        ),
        legend=dict(
            title_font_family="Arial, sans-serif",
            font=dict(size=11),
            bgcolor='rgba(255, 255, 255, 0.8)',
            bordercolor='gray',
            borderwidth=1
        )
    )
    
    # Update traces for better visibility
    fig.update_traces(
        marker=dict(size=10, line=dict(width=1, color='DarkSlateGrey')),
        opacity=0.7
    )
    
    # Update axes
    fig.update_xaxes(
        showgrid=True, 
        gridwidth=1, 
        gridcolor='rgba(128, 128, 128, 0.2)',
        title_font=dict(size=12, family='Arial, sans-serif')
    )
    fig.update_yaxes(
        showgrid=True, 
        gridwidth=1, 
        gridcolor='rgba(128, 128, 128, 0.2)',
        title_font=dict(size=12, family='Arial, sans-serif')
    )
    
    # Clean up facet labels to remove override
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    
    fig.write_html('../results/charts/interactive_scatter.html')
    fig.show()
    
    print("✓ Interactive chart saved to results/charts/interactive_scatter.html")

## 5. Statistical Summary for Report

In [ ]:
if not df_all.empty:
    # Generate comprehensive report
    report = []
    report.append("="*80)
    report.append("ML HALLUCINATION RESEARCH - STATISTICAL SUMMARY")
    report.append("="*80)
    report.append(f"\nTotal Experiments Conducted: {stats['total_experiments']}")
    report.append(f"Total Tests Performed: {stats['total_tests']}")
    report.append(f"Data Collection Period: {df_all['created_at'].min()} to {df_all['created_at'].max()}")
    
    report.append("\n" + "="*80)
    report.append("MITIGATION STRATEGY EFFECTIVENESS")
    report.append("="*80)
    
    for strategy in strategy_performance.index:
        row = strategy_performance.loc[strategy]
        report.append(f"\n{strategy.upper().replace('_', ' ')}:")
        report.append(f"  • Total Tests: {int(row['Total Tests'])}")
        report.append(f"  • Hallucinations Detected: {int(row['Hallucinations'])}")
        report.append(f"  • Hallucination Rate: {row['Hallucination Rate (%)']}%")
        report.append(f"  • Avg Response Time: {row['Avg Response Time (ms)']:.0f}ms")
        report.append(f"  • Avg Token Usage: {row['Avg Tokens']:.0f} tokens")
    
    # Calculate improvement over baseline
    if 'baseline' in strategy_performance.index:
        baseline_rate = strategy_performance.loc['baseline', 'Hallucination Rate (%)']
        report.append("\n" + "="*80)
        report.append("IMPROVEMENT OVER BASELINE")
        report.append("="*80)
        
        for strategy in strategy_performance.index:
            if strategy != 'baseline':
                rate = strategy_performance.loc[strategy, 'Hallucination Rate (%)']
                improvement = baseline_rate - rate
                pct_improvement = (improvement / baseline_rate * 100) if baseline_rate > 0 else 0
                report.append(f"\n{strategy.upper().replace('_', ' ')}:")
                report.append(f"  • Absolute Reduction: {improvement:.1f} percentage points")
                report.append(f"  • Relative Improvement: {pct_improvement:.1f}%")
    
    report.append("\n" + "="*80)
    report.append("TOP HALLUCINATION-PRONE CATEGORIES")
    report.append("="*80)
    
    top_categories = category_performance.nlargest(5, 'Rate (%)')
    for idx, (cat, row) in enumerate(top_categories.iterrows(), 1):
        report.append(f"\n{idx}. {cat.replace('_', ' ').title()}")
        report.append(f"   Rate: {row['Rate (%)']}% ({int(row['Hallucinations'])}/{int(row['Total'])} tests)")
    
    report.append("\n" + "="*80)
    
    # Print and save
    report_text = "\n".join(report)
    print(report_text)
    
    with open('../results/reports/statistical_summary.txt', 'w') as f:
        f.write(report_text)
    
    print("\n✓ Report saved to results/reports/statistical_summary.txt")

## 6. Export All Data

In [ ]:
if not df_all.empty:
    # Export complete dataset
    df_all.to_csv('../data/exports/complete_dataset.csv', index=False)
    print("✓ Complete dataset exported to data/exports/complete_dataset.csv")
    
    # Export summary tables
    strategy_performance.to_csv('../data/exports/strategy_summary.csv')
    category_performance.to_csv('../data/exports/category_summary.csv')
    
    print("✓ Summary tables exported")
    print("\nAll data ready for your report!")

## Key Insights for Your Report

Use this section to document your findings:

### 1. Primary Research Question
**Do mitigation strategies effectively reduce LLM hallucinations in cybersecurity contexts?**

Your finding:
- 

### 2. Most Effective Strategy
- Which strategy performed best?
- By how much did it reduce hallucinations?
- What were the trade-offs?

Your analysis:
- 

### 3. Vulnerability Categories
- Which prompt types were most prone to hallucination?
- Did this vary by mitigation strategy?

Your notes:
- 

### 4. Practical Recommendations
- For cybersecurity applications, which strategy would you recommend?
- When is each strategy most appropriate?

Your recommendations:
- 

### 5. Limitations & Future Work
- What are the limitations of this study?
- What would you do differently with more resources?

Your thoughts:
- 

In [ ]:
# Cleanup
db.close()
print("\n✓ Analysis complete!")
print("\nAll visualizations and reports are in:")
print("  - results/charts/")
print("  - results/reports/")
print("  - data/exports/")